# Why Agents?

What is an agent?

An agent is a reference to combining models that can perform some kind of reasoning, like large language models (e.g ChatGPT, Llama2, Mistral, etc...) with tools to give it access to the real world,
so they can do things like browsing the internet, buying stuff, etc...

Ok, so, why is there so much hype around agents right now?

Because Agents are cool! Recently with the advance of LLMs, we've seen them become an amazing tool to do all sorts of things like building apps, browse the internet and more.



SOme neat examples of these kinds of agents can be found in here:

- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)

Now, today although they seem extremely powerful, agents are still at a very early stage in terms of readiness to deploy as products, something you can atest by listening to Andrej Karpathy talk about agents in this talk here:

- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)

This live-training is all about! Getting you excited about this amazing new technology, understanding it from the ground up but with a focus on practical applications and fun stuff you can do with them! 

# What is an Agent?

An agent is nothing more than some entity that can _think_ and _act_, that's right, in a way you're an agent! 

After all you can think and act on those thoughts like in the case of coming to this live-training:

- Thought: "I want to learn about agents"

- Action: "Go to the internet and research cool platforms where I can learn about agents"

- Thought: "O'Reilly has some awesome courses and live-trainings"

- Action: "Look up O'Reilly courses"

- Thought: "Live-trainings by instructor Lucas are awesome"

- Action: "Schedule live-training about agents with instructor Lucas Soares" (lol)

In a way this is a simplified rendition of what brought you here, obviously not necessarily in this particular order nor these particular sets of thought and action pairs. This particular way of thinking about how to structure thoughts and actions is well represented in the paper: [ReACT](https://arxiv.org/pdf/2210.03629.pdf). 

With regards to LLMs, how can bring this idea to fruition thinking about the LLM model as the reasoning and thinking engine?

We can start simple and just call the openai API to start:

In [77]:
#pip install openai

In [78]:
# uncomment this if running locally
#!pip install python-dotenv
# from dotenv import load_dotenv

# load_dotenv()

import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [79]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful research and programming assistant"},
            {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Create a joke about a bald teacher explaining agents to wonderful students.")
Markdown(output)

Why did the bald teacher get excited about explaining agents to his wonderful students?

Because he knew a little "hair" and there could make a big difference in their learning!

Ok cool, so here we have three ideas of actions to perform:

- Creating directories
- Listing files
- Removing files

Let's transform them into functions that we could call just like in any type of Python-based application.

In [80]:
def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

Now, let's imagine that we wanted to create an agent that would perform these actions for us based on some input that we give it, how can we connect models that we know and can use today like ChatGPT, with these tools that do stuff in the real world?

To answer this question, how about we give a task to the model, and for that task we ask it to list the steps that it needs to perform to complete the task, and then for each of those steps we would ask the model to decide whether or not a function should be called to execute that task? 

In the famous paper ['Toolformer'](https://arxiv.org/pdf/2302.04761.pdf) they demonstrated that today's advanced LLMs like the gpt-series could teacha themselves how to properly call and use external tools!

Isn't that awesome???

So, let's see if we can hack our way into connecting the llm response with the functions that we want that llm to use.

In [81]:
def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content


Now, how can we actually put it all together so that given a task, a model can:

- Plan the task
- Execute actions to complete the task
- Know when to call a function


This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [82]:
def get_response(prompt_question, model="gpt-3.5-turbo-16k"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

    

task_description = "Create a folder called 'funny-pancakes-recipes'. Inside that folder, \
create a file called '3-funny-pancake-recipes.md"

prompt = f"""Given this task: {task_description}, \n
        Consider you have access to the following functions:
                            
    def create_directory(dir_name):
        os.makedirs(dir_name, exist_ok=True)

    def create_file():
        with open('test.txt', 'w'):
            pass

    def list_files():
        files = os.listdir()
        for file in files:
            print(file)
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """

output = get_response(prompt)

Markdown(output)

create_directory('funny-pancakes-recipes')

In [83]:
import os
# Remove any ```python or ``` tags from the output string if present
output_clean = output.replace("```python", "").replace("```", "").strip()
Markdown(output_clean)

create_directory('funny-pancakes-recipes')

In [84]:
exec(output_clean)

In [85]:
!ls -d ./* | grep pancakes

./funny-pancakes-recipes


At this point we can start identifying a lot of issues with this approach despite our early sucess:

- Uncertainty of model's outputs can affect our ability to reliably call the functions
- We need more structured ways to prepare the inputs of the function calls
- We need better ways to put everything together (just feeding the entire functions like this makes it a very clunky and non-scalable framework for more complex cases)

There are many more issues but starting with these, we can now look at frameworks and see how they fix these issues and with that in mind understand what is behind their implementations!

I personally think this is a much better way to understand what is going on behind agents in practice rather than just use the more higher level frameworks right of the bat!

# OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:
1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Below is an example taken from their official documentation:

In [86]:
from openai import OpenAI
import json

client = OpenAI()

Let's look at how our previous model with those three simple functions: `create_directory()`, `create_file()`, and `list_files()` would be implemented using OpenAI's function calling approach:

In [87]:
import json
import subprocess
import json
import os

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""
    os.mkdir(directory_name)
    return json.dumps({"directory_name": directory_name})


tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

tools = [tool_create_directory]

In [88]:
import json

def run_terminal_task():
    messages = [
        {"role": "user", 
         "content": "Create a folder called 'pancakes-are-better-than-waffles'."}]
    tools = [tool_create_directory]  
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

ChatCompletion(id='chatcmpl-D7hGtQHurauj0WIXNYMq5b0a0vovH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The folder named 'pancakes-are-better-than-waffles' has been created successfully!", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1770727003, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_f4ae844694', usage=CompletionUsage(completion_tokens=21, prompt_tokens=74, total_tokens=95, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [89]:
output.choices[0].message.content

"The folder named 'pancakes-are-better-than-waffles' has been created successfully!"

In [90]:
!ls -d */ | grep waffles

pancakes-are-better-than-waffles/


Great! We implemented openai function calling for creating directories! We could evolve this approach but let's stop for now.

See more info on these examples from OpenAI's [official cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

Now, let's implement the agent loop using a few LangChain components but implementing the loop logic ourselves!


In [91]:
from langchain_ollama import ChatOllama

llm_structured_output = ChatOllama(model="llama3.2", format="json", temperature=0)

llm_structured_output.invoke("Output names of cats!")

AIMessage(content='{ "Mittens":"A grey and white cat with bright green eyes", \n"Muffin": "A fluffy orange tabby with a sweet disposition",\n"Bella": "A sleek black cat with a shiny coat and playful personality",\n"Pussycat": "A mischievous calico cat with a love for adventure",\n"Salem": "A mysterious grey and white cat with piercing blue eyes" }', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-02-10T12:36:56.840690567Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11953234380, 'load_duration': 180106240, 'prompt_eval_count': 30, 'prompt_eval_duration': 97534463, 'eval_count': 85, 'eval_duration': 8688627410, 'model_name': 'llama3.2'}, id='run-9d49654d-72a5-4122-b0a3-fd06f824da23-0', usage_metadata={'input_tokens': 30, 'output_tokens': 85, 'total_tokens': 115})

In [92]:
import os

def create_dir(folder_path):
    """
    Creates a directory given a folder path.
    """
    # Normalize path - remove leading/trailing whitespace and leading slashes
    folder_path = folder_path.strip().lstrip('/')
    
    # Use makedirs to create nested directories, with exist_ok to avoid errors
    os.makedirs(folder_path, exist_ok=True)
    
    return f"Folder path was created at: {os.path.abspath(folder_path)}"


create_dir("lucas-test-local-agent")

'Folder path was created at: /home/belbut/repos/collab_lucas/oreilly_courses/oreilly_live_training_agents/notebooks/lucas-test-local-agent'

In [93]:
!ls -d */ | grep lucas-test-local-agent

lucas-test-local-agent/


In [94]:
def create_file(file_path, contents=""):
    """
    Creates a file with content.
    If no content is provided it will create an empty file.
    """
    # Normalize path - remove leading/trailing whitespace and leading slashes
    file_path = file_path.strip().lstrip('/')
    
    # Create parent directories if they don't exist
    parent_dir = os.path.dirname(file_path)
    if parent_dir:
        os.makedirs(parent_dir, exist_ok=True)
    
    with open(file_path, "w") as f:
        f.write(contents)
    
    return f"A file was created at: {os.path.abspath(file_path)}"

create_file("./lucas-test-local-agent/file-text.txt", "This is a test")

'A file was created at: /home/belbut/repos/collab_lucas/oreilly_courses/oreilly_live_training_agents/notebooks/lucas-test-local-agent/file-text.txt'

In [95]:
def read_file(file_path):
    """
    Reads from file given its path.
    """
    
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file("./lucas-test-local-agent/file-text.txt")

'This is a test'

In [96]:
tools = [create_dir, create_file, read_file]

llm_tools = llm_structured_output.bind_tools(tools)

In [97]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an agent that helps users with desktop tasks like reading and writing files and creating directories. You either call a tool
from the options available: create_dir, create_file or read_file, or you return a summary of the tasks performed and at the end a string: END.
"""
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])
llm_agent = prompt_template | llm_tools

In [98]:
llm_agent.invoke("Create file named lucas-loves-pancakes.md")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-02-10T12:37:08.473763372Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10704437948, 'load_duration': 179216421, 'prompt_eval_count': 297, 'prompt_eval_duration': 7150509937, 'eval_count': 25, 'eval_duration': 2506000310, 'model_name': 'llama3.2'}, id='run-bcb58d4f-d124-4969-8016-43c162b24592-0', tool_calls=[{'name': 'create_file', 'args': {'file_path': 'lucas-loves-pancakes.md'}, 'id': 'b5d3a991-aafc-4d2e-8e49-a45ca98825a6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 297, 'output_tokens': 25, 'total_tokens': 322})

In [99]:
output_tool_call = llm_agent.invoke("Create a file named ./test-agent.txt")
output_tool_call.tool_calls

[{'name': 'create_file',
  'args': {'file_path': './test-agent.txt'},
  'id': '4e437d82-4e17-403f-89cf-e9cf169fe2f9',
  'type': 'tool_call'}]

In [100]:
tool_mapping = {
    "create_file": create_file,
    "create_dir": create_dir,
    "read_file": read_file
}

In [101]:
def llm_call(query, observations=[], actions_taken=[]):
    """
    Calls the llm, it can return a tool call with arguments for calling different tools,
    or an output to the user.
    """
    template = """
    Imagine you are a simple assistant tasked with managing a file system. You have access to three tools:
    
    1. create_dir: Creates a new directory.
    2. create_file: Creates a new file and optionally writes content to it.
    3. read_file: Reads the contents of an existing file.

    Based on the user input and your observations, choose an action to execute. Your action must follow these guidelines:
    
    * Action Guidelines *
    1) Only one action is allowed per iteration.
    2) Be concise and specific about what to create, write, or read.
    3) Provide clear reasoning for your action.
    4) Always consider the current context before taking action.
    
    Respond in the following format:
    Thought: {{Your reasoning here}}
    Action: {{Action name with parameters}}
    
    Available Actions:
    - create_dir [directory_name]
    - create_file [file_name]; [content (optional)]
    - read_file [file_name]
    """

    # Prepare the input for the LLM
    prompt = f"""
    {template}

    User Query: {query}
    Observations: {observations}
    Actions Taken So Far: {actions_taken}
    """

    output = llm_agent.invoke(prompt)

    if output.tool_calls:
        for tool_call in output.tool_calls:
            function_name = tool_call["name"]
            function_args = tool_call.get("args", [])  # default to empty list if "args" is missing

            # Check if function_name exists in tool_mapping
            if function_name not in tool_mapping:
                print(f"Error: Unknown function name '{function_name}'")
                continue

            try:
                # Attempt to call the function with the provided arguments
                tool_output = tool_mapping[function_name](**function_args)
                actions_taken.append(function_name)
                observations.append(tool_output)
            except TypeError as e:
                # Handle errors if the number of arguments does not match the expected number for the function
                print(f"Error: Invalid arguments for function '{function_name}': {e}")
    
    
    return output, actions_taken, observations

In [102]:
llm_call("Create a folder named: lucas-tests-tool-calling")

(AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-02-10T12:37:22.462296256Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10641588281, 'load_duration': 278449368, 'prompt_eval_count': 522, 'prompt_eval_duration': 6945756444, 'eval_count': 24, 'eval_duration': 2555880465, 'model_name': 'llama3.2'}, id='run-e7e9efb9-511f-4b14-9abf-e9460afabeba-0', tool_calls=[{'name': 'create_dir', 'args': {'folder_path': 'lucas-tests-tool-calling'}, 'id': 'a74da744-0bfd-4498-8968-dcc51c2949c2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 522, 'output_tokens': 24, 'total_tokens': 546}),
 ['create_dir'],
 ['Folder path was created at: /home/belbut/repos/collab_lucas/oreilly_courses/oreilly_live_training_agents/notebooks/lucas-tests-tool-calling'])

In [103]:
llm_call("Create a file at: ./lucas-tests-tool-calling/test1.txt with the contents: Hello Lucas! You just made a tool call!")

(AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-02-10T12:37:31.262643779Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8790495836, 'load_duration': 178568632, 'prompt_eval_count': 579, 'prompt_eval_duration': 2537080275, 'eval_count': 42, 'eval_duration': 4558291713, 'model_name': 'llama3.2'}, id='run-ce4185ab-339f-4215-8d02-a446b80f3708-0', tool_calls=[{'name': 'create_file', 'args': {'contents': 'Hello Lucas! You just made a tool call!', 'file_path': './lucas-tests-tool-calling/test1.txt'}, 'id': 'b93e29d2-28f2-4fd6-8d0c-7a1b1f39a4b5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 579, 'output_tokens': 42, 'total_tokens': 621}),
 ['create_dir', 'create_file'],
 ['Folder path was created at: /home/belbut/repos/collab_lucas/oreilly_courses/oreilly_live_training_agents/notebooks/lucas-tests-tool-calling',
  'A file was created at: /home/belbut/repos/collab_lucas/oreilly_courses/oreilly_live_training_agents/not

In [104]:
llm_call("Read the file contents from  ./lucas-tests-tool-calling/test1.txt")

(AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-02-10T12:37:38.766234476Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7494088299, 'load_duration': 155881515, 'prompt_eval_count': 612, 'prompt_eval_duration': 3643868850, 'eval_count': 27, 'eval_duration': 2755000035, 'model_name': 'llama3.2'}, id='run-edede90b-2d62-4da4-bfc7-7fce29441578-0', tool_calls=[{'name': 'read_file', 'args': {'file_path': './lucas-tests-tool-calling/test1.txt'}, 'id': 'ce219557-030c-4c25-88d8-5bfdbd0ca972', 'type': 'tool_call'}], usage_metadata={'input_tokens': 612, 'output_tokens': 27, 'total_tokens': 639}),
 ['create_dir', 'create_file', 'read_file'],
 ['Folder path was created at: /home/belbut/repos/collab_lucas/oreilly_courses/oreilly_live_training_agents/notebooks/lucas-tests-tool-calling',
  'A file was created at: /home/belbut/repos/collab_lucas/oreilly_courses/oreilly_live_training_agents/notebooks/lucas-tests-tool-calling/test1.txt',


In [105]:
def agent_loop(query):
    iter_count = 0
    obs = []
    acts_taken = []
    while True:
        output,obs,acts_taken = llm_call(query, obs, acts_taken)
        print(output)
        iter_count+=1
        if iter_count>=3:
            print(f"Breaking after {iter_count} iterations")
            break
        if output.content!="":
            break
    
    return output.content


agent_loop("Create a folder in current directory named 'testing-multiple-calls' and inside that folder create a file named nested-file.txt")        

content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2026-02-10T12:37:43.91256023Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5133883725, 'load_duration': 159422885, 'prompt_eval_count': 536, 'prompt_eval_duration': 1581026333, 'eval_count': 23, 'eval_duration': 2510950361, 'model_name': 'llama3.2'} id='run-fb04a12a-62ba-4436-b4bf-32dbb5b49414-0' tool_calls=[{'name': 'create_dir', 'args': {'folder_path': '/testing-multiple-calls'}, 'id': 'bf1b90ae-0911-4a0b-8cd7-5c989a9db64d', 'type': 'tool_call'}] usage_metadata={'input_tokens': 536, 'output_tokens': 23, 'total_tokens': 559}
content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2026-02-10T12:37:52.355147475Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8440195130, 'load_duration': 245708903, 'prompt_eval_count': 575, 'prompt_eval_duration': 1651546785, 'eval_count': 47, 'eval_duration': 4791053288, 'model_name': 'llama3.2'} id='run-a2e4d6f3-2d

''

# Cleanup - Remove all test files and directories created by this notebook
Run this cell to clean up all directories and files created during the notebook execution.

In [106]:
import shutil
import os

# List of directories and files created by this notebook
items_to_remove = [
    'pancakes-are-better-than-waffles',
    'lucas-test-local-agent',
    'testing-multiple-calls',
    'test-folder',
    'test-agent-folder',
    'agent-test',
    'test-dir',
    'my-test-directory',
    'test_output',
    'output',
    'example_output',
]

removed_items = []
not_found_items = []

for item in items_to_remove:
    if os.path.exists(item):
        if os.path.isdir(item):
            shutil.rmtree(item)
            removed_items.append(f"Directory: {item}")
        else:
            os.remove(item)
            removed_items.append(f"File: {item}")
    else:
        not_found_items.append(item)

# Also remove any .txt files created in test directories
for root, dirs, files in os.walk('.'):
    # Skip hidden directories and common non-test directories
    if any(skip in root for skip in ['.git', '.ipynb_checkpoints', '__pycache__', '.venv', 'venv']):
        continue
    for file in files:
        if file.endswith('.txt') and any(test in root for test in ['test', 'lucas', 'pancake', 'agent', 'output']):
            file_path = os.path.join(root, file)
            try:
                os.remove(file_path)
                removed_items.append(f"File: {file_path}")
            except:
                pass

print("\n=== Cleanup Complete ===" )
if removed_items:
    print(f"\nRemoved {len(removed_items)} item(s):" )
    for item in removed_items:
        print(f"  - {item}")
else:
    print("\nNo items to remove.")

if not_found_items:
    print(f"\n{len(not_found_items)} item(s) not found (already cleaned or never created):" )
    for item in not_found_items:
        print(f"  - {item}")


=== Cleanup Complete ===

Removed 5 item(s):
  - Directory: pancakes-are-better-than-waffles
  - Directory: lucas-test-local-agent
  - Directory: testing-multiple-calls
  - File: ./lucas-tests-tool-calling/test1.txt
  - File: ./legacy-notebooks/lucas-test-agents/file-text.txt

8 item(s) not found (already cleaned or never created):
  - test-folder
  - test-agent-folder
  - agent-test
  - test-dir
  - my-test-directory
  - test_output
  - output
  - example_output


# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)